In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [75]:
pd.set_option("display.max_columns", None)   # show all columns
pd.set_option("display.max_rows", None)      # show all rows if needed

In [76]:
# CLEANUP DATA

# Read file, skipping the first two metadata rows
df = pd.read_csv("SquishyData.csv", skiprows=[1,2])

# Drop the first 17 columns with unneeded data
df = df.iloc[:, 17:]

# Drop row 12, because of missing data and reset the indexing
df = df.drop(index=12)
df = df.reset_index(drop=True)

In [77]:
# remove all columns except UEQ columns
df_UEQ = df.filter(regex=r'^UEQ', axis=1)

# Invert the scale from 1-7 to -3 to +3
df_UEQ = (df_UEQ - 4).astype(int)

# Re-Attatch the PID
df_UEQ.insert(0, "PID", range(1, len(df_UEQ) + 1))

In [78]:
# List of items to invert
items_to_invert = [1, 2, 6, 7, 8, 11, 13, 14, 15, 16, 20, 22, 26]

# find columns that match those item numbers
cols_to_invert = [col for col in df_UEQ.columns 
                  if col.split('_')[-1].isdigit() and int(col.split('_')[-1]) in items_to_invert]

# invert these columns
df_UEQ[cols_to_invert] = df_UEQ[cols_to_invert] * -1

df_UEQ

,PID,UEQ 1 (Standard)_1,UEQ 1 (Standard)_2,UEQ 1 (Standard)_3,UEQ 1 (Standard)_4,UEQ 1 (Standard)_5,UEQ 1 (Standard)_6,UEQ 1 (Standard)_7,UEQ 1 (Standard)_8,UEQ 1 (Standard)_9,UEQ 1 (Standard)_10,UEQ 1 (Standard)_11,UEQ 1 (Standard)_12,UEQ 1 (Standard)_13,UEQ 1 (Standard)_14,UEQ 1 (Standard)_15,UEQ 1 (Standard)_16,UEQ 1 (Standard)_17,UEQ 1 (Standard)_18,UEQ 1 (Standard)_19,UEQ 1 (Standard)_20,UEQ 1 (Standard)_21,UEQ 1 (Standard)_22,UEQ 1 (Standard)_23,UEQ 1 (Standard)_24,UEQ 1 (Standard)_25,UEQ 1 (Standard)_26,UEQ 2 (Standard)_1,UEQ 2 (Standard)_2,UEQ 2 (Standard)_3,UEQ 2 (Standard)_4,UEQ 2 (Standard)_5,UEQ 2 (Standard)_6,UEQ 2 (Standard)_7,UEQ 2 (Standard)_8,UEQ 2 (Standard)_9,UEQ 2 (Standard)_10,UEQ 2 (Standard)_11,UEQ 2 (Standard)_12,UEQ 2 (Standard)_13,UEQ 2 (Standard)_14,UEQ 2 (Standard)_15,UEQ 2 (Standard)_16,UEQ 2 (Standard)_17,UEQ 2 (Standard)_18,UEQ 2 (Standard)_19,UEQ 2 (Standard)_20,UEQ 2 (Standard)_21,UEQ 2 (Standard)_22,UEQ 2 (Standard)_23,UEQ 2 (Standard)_24,UEQ 2 (Standard)_25,UEQ 2 (Standard)_26,UEQ 1 (Squishy)_1,UEQ 1 (Squishy)_2,UEQ 1 (Squishy)_3,UEQ 1 (Squishy)_4,UEQ 1 (Squishy)_5,UEQ 1 (Squishy)_6,UEQ 1 (Squishy)_7,UEQ 1 (Squishy)_8,UEQ 1 (Squishy)_9,UEQ 1 (Squishy)_10,UEQ 1 (Squishy)_11,UEQ 1 (Squishy)_12,UEQ 1 (Squishy)_13,UEQ 1 (Squishy)_14,UEQ 1 (Squishy)_15,UEQ 1 (Squishy)_16,UEQ 1 (Squishy)_17,UEQ 1 (Squishy)_18,UEQ 1 (Squishy)_19,UEQ 1 (Squishy)_20,UEQ 1 (Squishy)_21,UEQ 1 (Squishy)_22,UEQ 1 (Squishy)_23,UEQ 1 (Squishy)_24,UEQ 1 (Squishy)_25,UEQ 1 (Squishy)_26,UEQ 2 (Squishy)_1,UEQ 2 (Squishy)_2,UEQ 2 (Squishy)_3,UEQ 2 (Squishy)_4,UEQ 2 (Squishy)_5,UEQ 2 (Squishy)_6,UEQ 2 (Squishy)_7,UEQ 2 (Squishy)_8,UEQ 2 (Squishy)_9,UEQ 2 (Squishy)_10,UEQ 2 (Squishy)_11,UEQ 2 (Squishy)_12,UEQ 2 (Squishy)_13,UEQ 2 (Squishy)_14,UEQ 2 (Squishy)_15,UEQ 2 (Squishy)_16,UEQ 2 (Squishy)_17,UEQ 2 (Squishy)_18,UEQ 2 (Squishy)_19,UEQ 2 (Squishy)_20,UEQ 2 (Squishy)_21,UEQ 2 (Squishy)_22,UEQ 2 (Squishy)_23,UEQ 2 (Squishy)_24,UEQ 2 (Squishy)_25,UEQ 2 (Squishy)_26
0,1,2,-3,2,-3,2,2,2,-2,-1,2,-2,0,-3,-1,3,-1,2,2,-3,1,-3,-3,-3,1,-1,2,-1,-2,2,-3,-1,2,2,-2,-3,3,0,0,-2,0,3,-1,-1,2,-2,3,-2,-2,-2,0,0,2,-3,-2,-2,-3,-1,-2,-2,-1,-2,-2,-2,-2,-3,-2,-2,-3,-2,-3,1,-1,-3,-1,-3,0,-3,-2,-3,-3,-2,-3,-2,-3,-3,-2,-3,-2,-2,-3,-3,-2,-2,-2,-2,-3,-3,-1,-3,-1,-2,-1,-2,-2
1,2,-1,-2,1,-1,-1,1,1,-2,-2,-2,0,-1,-1,0,-1,-1,-1,-1,-1,-3,-3,0,-1,-2,-3,-1,-2,-3,-1,-2,-2,-3,-3,1,-3,1,-1,-1,-2,-1,1,-2,-1,1,0,-1,-3,-2,-1,0,-1,-1,-2,-3,-3,-3,-3,-2,-3,1,-3,2,-2,-3,-3,-3,-3,-3,-2,-2,-1,-2,-2,-2,-2,-3,-3,-2,-2,-2,-2,-2,-1,0,-1,1,-2,-2,1,-1,-1,0,-1,-1,-1,-1,-1,-2,0,-1,-1,-1,-1,-1
2,3,-1,0,1,-1,0,-1,-2,-2,1,1,-3,0,-2,-2,0,-1,1,-1,0,-1,-3,-2,-2,-1,-1,-2,-2,-2,-2,-2,1,-1,-1,1,-1,0,-3,-3,-3,-1,-1,-2,-1,-2,0,-2,-2,-3,-3,-3,-3,-2,-3,-3,-3,-3,-1,-2,-3,-1,0,-2,-1,-2,-3,-3,-3,-3,-1,-1,0,-1,-3,-2,-2,-3,-3,-3,-3,-3,-3,-3,-2,-3,-3,-2,1,-3,-2,-3,-3,-3,-3,-3,-1,-1,3,-1,-3,-3,-3,-3,-3,-3
3,4,0,-3,1,-3,1,1,1,-3,-3,2,-1,-1,-3,0,1,-1,0,0,-2,-3,-3,-3,1,0,-1,2,1,-2,1,-3,0,1,1,-3,-3,2,0,-1,-3,0,1,0,0,0,-3,-2,-2,-2,0,1,0,1,-2,-2,-3,-2,0,-2,-1,-3,0,-3,-1,-1,-2,-3,-3,-3,-1,-2,-3,-1,0,-1,-3,-2,-3,-3,-3,-3,-2,-2,-1,-1,-1,-3,-2,-3,0,-1,-2,-3,-3,-2,-2,-2,-3,-1,-2,-1,-2,-2,-3,-3
4,5,-2,-2,-1,-2,-2,0,-1,1,-2,-2,-1,-2,-2,-2,-1,-1,-1,-1,-2,-1,-1,-2,-1,-2,-1,-1,-2,-3,-2,-2,-2,-2,-3,-1,-3,-1,-2,-1,-1,-1,-1,-2,-1,-2,-2,-1,-1,-1,-1,-2,-1,-1,-1,1,-1,1,-2,-2,-1,-1,-1,-2,0,-1,1,0,-1,0,0,0,0,1,1,1,1,-1,-1,0,-1,-2,-2,-1,-2,-1,-2,1,-1,-2,1,-2,-1,-1,-2,-1,0,-1,1,-1,1,-1,-1,-2,-1,-1
5,6,-3,-3,-1,-3,-2,-1,-2,0,0,-1,-1,-2,-2,-2,-2,-2,-1,-2,-1,-2,-2,0,-1,-2,-2,2,-3,-3,-3,-3,-3,-2,-2,-1,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-2,-3,-3,-2,-3,-3,-3,-2,-1,-2,-1,0,2,0,1,0,1,1,1,1,-1,1,1,1,1,0,2,1,1,-1,1,2,2,1,-2,-2,-1,-1,0,0,1,0,1,2,-1,-1,-2,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0
6,7,0,-3,1,-3,-1,1,1,0,0,3,-3,0,-3,0,3,0,0,0,-3,0,-3,-3,0,0,1,0,-3,-3,3,-3,-2,0,0,0,0,0,0,0,-3,0,3,0,-3,0,-3,0,-3,-3,0,3,3,3,-3,-3,-3,-3,-3,-3,-3,0,0,-2,0,-3,-3,-1,-2,-3,-3,-3,0,0,-2,-2,0,-3,-3,-3,-3,-3,-3,-3,-3,-1,-3,-1,0,-2,0,-3,-3,0,-3,-3,-3,0,-2,-3,-3,-2,-3,-3,

In [81]:
# Calculate mean of the two tasks for each question
for i in range(1, 27):  # for all 26 UEQ items
    df_UEQ[f'UEQ (Standard)_{i}_mean'] = df_UEQ[[f'UEQ 1 (Standard)_{i}', f'UEQ 2 (Standard)_{i}']].mean(axis=1)
    df_UEQ[f'UEQ (Squishy)_{i}_mean'] = df_UEQ[[f'UEQ 1 (Squishy)_{i}', f'UEQ 2 (Squishy)_{i}']].mean(axis=1)

# Keep PID and all columns that have "_mean" in their name
columns_to_keep = ['PID'] + [col for col in df_UEQ.columns if '_mean' in col]

# Create a new, smaller DataFrame
df_means = df_UEQ[columns_to_keep]


# Keep PID at the front
cols = ['PID']

# Get all Standard columns
standard_cols = [col for col in df_means.columns if 'Standard' in col]

# Get all Squishy columns
squishy_cols = [col for col in df_means.columns if 'Squishy' in col]

# Combine in the desired order
df_means = df_means[cols + standard_cols + squishy_cols]

df_means

,PID,UEQ (Standard)_1_mean,UEQ (Standard)_2_mean,UEQ (Standard)_3_mean,UEQ (Standard)_4_mean,UEQ (Standard)_5_mean,UEQ (Standard)_6_mean,UEQ (Standard)_7_mean,UEQ (Standard)_8_mean,UEQ (Standard)_9_mean,UEQ (Standard)_10_mean,UEQ (Standard)_11_mean,UEQ (Standard)_12_mean,UEQ (Standard)_13_mean,UEQ (Standard)_14_mean,UEQ (Standard)_15_mean,UEQ (Standard)_16_mean,UEQ (Standard)_17_mean,UEQ (Standard)_18_mean,UEQ (Standard)_19_mean,UEQ (Standard)_20_mean,UEQ (Standard)_21_mean,UEQ (Standard)_22_mean,UEQ (Standard)_23_mean,UEQ (Standard)_24_mean,UEQ (Standard)_25_mean,UEQ (Standard)_26_mean,UEQ (Squishy)_1_mean,UEQ (Squishy)_2_mean,UEQ (Squishy)_3_mean,UEQ (Squishy)_4_mean,UEQ (Squishy)_5_mean,UEQ (Squishy)_6_mean,UEQ (Squishy)_7_mean,UEQ (Squishy)_8_mean,UEQ (Squishy)_9_mean,UEQ (Squishy)_10_mean,UEQ (Squishy)_11_mean,UEQ (Squishy)_12_mean,UEQ (Squishy)_13_mean,UEQ (Squishy)_14_mean,UEQ (Squishy)_15_mean,UEQ (Squishy)_16_mean,UEQ (Squishy)_17_mean,UEQ (Squishy)_18_mean,UEQ (Squishy)_19_mean,UEQ (Squishy)_20_mean,UEQ (Squishy)_21_mean,UEQ (Squishy)_22_mean,UEQ (Squishy)_23_mean,UEQ (Squishy)_24_mean,UEQ (Squishy)_25_mean,UEQ (Squishy)_26_mean
0,1,0.5,-2.5,2.0,-3.0,0.5,2.0,2.0,-2.0,-2.0,2.5,-1.0,0.0,-2.5,-0.5,3.0,-1.0,0.5,2.0,-2.5,2.0,-2.5,-2.5,-2.5,0.5,-0.5,2.0,-3.0,-2.5,-2.0,-3.0,-1.5,-2.5,-2.5,-1.5,-2.5,-2.0,-2.0,-2.5,-3.0,-2.0,-2.0,-2.5,-2.0,-3.0,-1.0,-1.0,-3.0,-1.0,-2.5,-0.5,-2.5,-2.0
1,2,-1.5,-2.5,0.0,-1.5,-1.5,-1.0,-1.0,-0.5,-2.5,-0.5,-0.5,-1.0,-1.5,-0.5,0.0,-1.5,-1.0,0.0,-0.5,-2.0,-3.0,-1.0,-1.0,-1.0,-2.0,-1.0,-2.0,-2.5,-2.5,-2.5,-2.0,-1.0,-2.0,1.0,-2.5,0.0,-0.5,-2.0,-2.0,-1.5,-2.0,-2.0,-1.5,-1.5,-1.0,-2.0,-1.0,-1.5,-1.5,-2.0,-2.0,-1.5
2,3,-1.5,-1.0,-0.5,-1.5,0.5,-1.0,-1.5,-0.5,0.0,0.5,-3.0,-1.5,-2.5,-1.5,-0.5,-1.5,0.0,-1.5,0.0,-1.5,-2.5,-2.5,-2.5,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-1.5,-2.5,-3.0,-1.5,0.5,-2.5,-1.5,-2.5,-3.0,-3.0,-3.0,-3.0,-1.0,-1.0,1.5,-1.0,-3.0,-2.5,-2.5,-3.0,-3.0,-3.0
3,4,0.5,-2.5,1.0,-3.0,0.5,1.0,1.0,-3.0,-3.0,2.0,-0.5,-1.0,-3.0,0.0,1.0,-0.5,0.0,0.0,-2.5,-2.5,-2.5,-2.5,0.5,0.5,-0.5,1.5,-2.5,-2.5,-2.5,-2.0,-0.5,-1.5,-1.0,-3.0,-1.0,-3.0,-0.5,-1.0,-2.0,-3.0,-3.0,-2.5,-1.5,-2.0,-3.0,-1.0,-1.0,-1.0,-2.5,-2.0,-3.0,-3.0
4,5,-2.0,-2.5,-1.5,-2.0,-2.0,-1.0,-2.0,0.0,-2.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.0,-1.5,-1.0,-1.5,-2.0,-1.0,-1.0,-1.5,-1.0,-2.0,-1.0,-1.0,-1.0,-0.5,-1.5,0.0,-2.0,-1.5,-1.5,0.0,-1.0,-2.0,0.5,-1.5,0.0,-0.5,-1.5,-0.5,0.0,-0.5,0.5,0.0,1.0,0.0,0.0,-1.5,-1.0,-0.5
5,6,-3.0,-3.0,-2.0,-3.0,-2.5,-1.5,-2.0,-0.5,-1.5,-2.0,-2.0,-2.5,-2.5,-2.5,-2.5,-2.5,-2.0,-2.5,-1.5,-2.5,-2.5,-1.0,-2.0,-2.5,-2.5,0.0,-1.5,-2.0,-1.0,-0.5,1.0,0.0,1.0,0.0,1.0,1.5,0.0,0.0,-1.5,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.5,-0.5,0.5,1.0,1.0,0.5
6,7,-1.5,-3.0,2.0,-3.0,-1.5,0.5,0.5,0.0,0.0,1.5,-1.5,0.0,-3.0,0.0,3.0,0.0,-1.5,0.0,-3.0,0.0,-3.0,-3.0,0.0,1.5,2.0,1.5,-3.0,-3.0,-3.0,-3.0,-3.0,-2.0,-3.0,-0.5,0.0,-2.0,0.0,-3.0,-3.0,-0.5,-2.5,-3.0,-3.0,-1.5,-1.0,-1.5,-2.5,-2.0,-1.5,-3.0,-3.0,-3.0
7,8,0.0,-1.0,2.0,-1.5,0.0,1.5,1.5,0.0,-1.0,2.0,0.0,0.5,-1.0,0.5,2.0,0.0,0.0,0.5,0.5,-0.5,-1.0,0.0,-2.0,1.5,0.5,2.0,-2.0,-2.0,-2.0,-3.0,-1.0,-1.0,-1.0,-2.5,-1.5,-2.0,-1.5,-2.0,-3.0,-1.0,-2.0,-2.5,-1.0,-0.5,-2.0,-0.5,-2.5,-1.5,-2.0,-1.0,-2.0,-1.0
8,9,-2.0,-2.0,-1.0,-0.5,0.0,-1.5,-1.0,0.0,-0.5,0.0,-0.5,-1.0,-0.5,-0.5,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,-1.5,0.0,-0.5,-2.0,-1.5,-2.0,0.0,-0.5,-2.0,-1.0,0.0,0.0,-1.5,-1.5,-1.0,-1.5,-1.0,-1.5,-1.5,-0.5,-1.0,0.5,0.0,-1.0,-0.5,-1.0,-1.0,-2.0,-1.5
9,10,-1.0,-1.5,-0.5,-2.0,-1.0,-1.0,-0.5,1.0,-2.0,-0.5,0.0,-1.5,-0.5,-0.5,-1.0,-1.0,-0.5,0.0,-1.5,-1.0,-0.5,0.0,-1.5,-1.0,-0.5,-1.0,-1.5,-0.5,1.0,-1.0,0.0,-0.5,0.0,-0.5,-2.0,0.5,0.0,-1.0,-1.0,-0.5,-1.5,-0.5,-0.5,-1.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0
